1. Подберите скорость обучения (eta) и количество итераций

In [1]:
import numpy as np

In [2]:
def calc_mae(y, y_pred):
    err = np.mean(np.abs(y - y_pred))
    return err

def calc_mse(y, y_pred):
    err = np.mean((y - y_pred)**2)
    return err

In [3]:
X = np.array([[ 1,  1],
              [ 1,  1],
              [ 1,  2],
              [ 1,  5],
              [ 1,  3],
              [ 1,  0],
              [ 1,  5],
              [ 1, 10],
              [ 1,  1],
              [ 1,  2]])
y = [45, 55, 50, 55, 60, 35, 75, 80, 50, 60]

In [4]:
n = X.shape[0]

eta = 1e-2    # нет ИЗМЕНЕНИЙ: оставила 0,01, так как значения MSE меняются плавно, не перескакивают
n_iter = 1200 # ИЗМЕНЕНИЯ: (было 100) на итерации 1150 значения MSE перестают значимо меняться,
              #             оставила ещё 50 итераций, чтобы убедиться в этом
W = np.array([1, 0.5])
print(f'Number of objects = {n} \
       \nLearning rate = {eta} \
       \nInitial weights = {W} \n')

for i in range(n_iter):
    y_pred = np.dot(X, W)
    err = calc_mse(y, y_pred)
    for k in range(W.shape[0]):
        W[k] -= eta * (1/n * 2 * X[:, k] @ (y_pred - y))
    if i % 10 == 0:
        eta /= 1.1
        print(f'Iteration #{i}: W_new = {W}, MSE = {round(err, 2)}')

Number of objects = 10        
Learning rate = 0.01        
Initial weights = [1.  0.5] 

Iteration #0: W_new = [2.08 4.27], MSE = 3047.75
Iteration #10: W_new = [ 6.67106886 10.61676385], MSE = 749.71
Iteration #20: W_new = [ 9.49320908 10.25731657], MSE = 648.91
Iteration #30: W_new = [11.85740092  9.83349244], MSE = 570.46
Iteration #40: W_new = [13.86876921  9.46898661], MSE = 508.03
Iteration #50: W_new = [15.59085668  9.15672679], MSE = 457.73
Iteration #60: W_new = [17.07337653  8.88789585], MSE = 416.77
Iteration #70: W_new = [18.35601294  8.65530964], MSE = 383.06
Iteration #80: W_new = [19.47073522  8.45317196], MSE = 355.08
Iteration #90: W_new = [20.44350656  8.27677488], MSE = 331.65
Iteration #100: W_new = [21.29557245  8.12226587], MSE = 311.9
Iteration #110: W_new = [22.044442    7.98646992], MSE = 295.12
Iteration #120: W_new = [22.7046421   7.86675281], MSE = 280.78
Iteration #130: W_new = [23.2883015   7.76091518], MSE = 268.46
Iteration #140: W_new = [23.80560705  7

2*. В этом коде мы избавляемся от итераций по весам, но тут есть ошибка, исправьте ее

In [5]:
n = X.shape[0]

eta = 1e-2 
n_iter = 100

W = np.array([1, 0.5])
print(f'Number of objects = {n} \
       \nLearning rate = {eta} \
       \nInitial weights = {W} \n')

for i in range(n_iter):
    y_pred = np.dot(X, W)
    err = calc_mse(y, y_pred)
#     for k in range(W.shape[0]):
#         W[k] -= eta * (1/n * 2 * X[:, k] @ (y_pred - y))
    # ИЗМЕНЕНИЯ
    # я уточнила столбец в матрице X, чтобы удовлетворить условию уммножения матрицы на вектор (количесво столбцов равно количеству строк)    
    W -= eta * (1/n * 2 * np.dot(X[:, 1], y_pred - y))
    # ИЗМЕНЕНИЯ
    # добавила строку с изменением шага(eta) на каждой интерации (i) в цикле for
    eta /= 1.1
    if i % 10 == 0:
        print(f'Iteration #{i}: W_new = {W}, MSE = {round(err,2)}')

Number of objects = 10        
Learning rate = 0.01        
Initial weights = [1.  0.5] 

Iteration #0: W_new = [4.77 4.27], MSE = 3047.75
Iteration #10: W_new = [10.10145365  9.60145365], MSE = 624.54
Iteration #20: W_new = [10.30592861  9.80592861], MSE = 613.07
Iteration #30: W_new = [10.3429761  9.8429761], MSE = 611.47
Iteration #40: W_new = [10.35382943  9.85382943], MSE = 611.02
Iteration #50: W_new = [10.35760203  9.85760203], MSE = 610.87
Iteration #60: W_new = [10.35900004  9.85900004], MSE = 610.82
Iteration #70: W_new = [10.35953089  9.85953089], MSE = 610.8
Iteration #80: W_new = [10.35973435  9.85973435], MSE = 610.79
Iteration #90: W_new = [10.35981262  9.85981262], MSE = 610.79


3*. Вместо того, чтобы задавать количество итераций, задайте другое условие останова алгоритма - когда веса перестают изменяться меньше определенного порога $\epsilon$.

In [19]:
# следующий код я составила заново, опираясь на материалы урока, используя данные из предыдущих заданий

In [6]:
display(X, y)

array([[ 1,  1],
       [ 1,  1],
       [ 1,  2],
       [ 1,  5],
       [ 1,  3],
       [ 1,  0],
       [ 1,  5],
       [ 1, 10],
       [ 1,  1],
       [ 1,  2]])

[45, 55, 50, 55, 60, 35, 75, 80, 50, 60]

In [18]:
Y = np.array(y)
W = np.array([1, 0.5])
eta = 1e-2

weight_dist = np.inf
min_weight_dist = 1e-4
iter_num = 0

print(f'Number of objects = {X.shape[0]} \
       \nLearning rate = {eta} \
       \nInitial weights = {W} \n')

while weight_dist > min_weight_dist:
    Y_pred = np.dot(X, W)
    dQ = 2 / X.shape[0] * np.dot(X.T, Y_pred - Y)
    W_new = W - eta * dQ 
    weight_dist = np.linalg.norm(W - W_new, ord=2)
    mse_err = calc_mse(Y, Y_pred)
   
    W = W_new
    iter_num +=1
    
    if iter_num % 10 == 0:         
        print(f'Iter {iter_num}: mse_error - {mse_err}, weights: {W}')

Number of objects = 10        
Learning rate = 0.01        
Initial weights = [1.  0.5] 

Iter 10: mse_error - 752.0184356756685, weights: [ 6.64172205 10.62940003]
Iter 20: mse_error - 632.7150870310963, weights: [10.02900674 10.16329008]
Iter 30: mse_error - 534.1424546924877, weights: [13.09636548  9.60903915]
Iter 40: mse_error - 452.07375384905697, weights: [15.89487851  9.10159809]
Iter 50: mse_error - 383.7456343612788, weights: [18.44838865  8.63855875]
Iter 60: mse_error - 326.857541015558, weights: [20.77834901  8.21605636]
Iter 70: mse_error - 279.4940955004128, weights: [22.90433054  7.83054239]
Iter 80: mse_error - 240.06060706491303, weights: [24.84419078  7.47877865]
Iter 90: mse_error - 207.22938050865514, weights: [26.61422391  7.15781043]
Iter 100: mse_error - 179.89501370640295, weights: [28.22929764  6.86494171]
Iter 110: mse_error - 157.13717956944907, weights: [29.70297804  6.59771249]
Iter 120: mse_error - 138.189639340991, weights: [31.04764353  6.35387814]
Iter